In [2]:
import sys
sys.path.append("/home/ocb/HardDrive_4TB/EGM/PHX/PhotoFITT/photofitt/")
from utils.normalisation import normalise_phc_timelapse
import tifffile
from tifffile import imsave
import os
import numpy as np
import cv2
import czifile as zis
import shutil
from skimage import io
from skimage import exposure


def normalise_data(Source_QC_folder, Target_QC_folder, Normalisation_QC_source, Normalisation_QC_target, Im_path):
    if Normalisation_QC_source == "Contrast stretching":

        for filename in os.listdir(Source_QC_folder):

            img = io.imread(os.path.join(Source_QC_folder,filename)).astype(np.float32)
            short_name = os.path.splitext(filename)

            p2, p99 = np.percentile(img, (1., 99.9))
            img = exposure.rescale_intensity(img, in_range=(p2, p99))

            img = 255 * img # Now scale by 255
            img = img.astype(np.uint8)
            cv2.imwrite(os.path.join(Im_path, "A", "test", f"{short_name[0]}.png"), img)

    if Normalisation_QC_target == "Contrast stretching":
        for filename in os.listdir(Target_QC_folder):

            img = io.imread(os.path.join(Target_QC_folder,filename)).astype(np.float32)
            short_name = os.path.splitext(filename)

            p2, p99 = np.percentile(img, (1., 99.9))
            img = exposure.rescale_intensity(img, in_range=(p2, p99))

            img = 255 * img # Now scale by 255
            img = img.astype(np.uint8)
            cv2.imwrite(os.path.join(Im_path, "B", "test", f"{short_name[0]}.png"), img)

    if Normalisation_QC_source == "Adaptive Equalization":
        for filename in os.listdir(Source_QC_folder):

            img = io.imread(os.path.join(Source_QC_folder,filename))
            short_name = os.path.splitext(filename)

            img = exposure.equalize_adapthist(img, clip_limit=0.03)

            img = 255 * img # Now scale by 255
            img = img.astype(np.uint8)
            cv2.imwrite(os.path.join(Im_path, "A", "test", f"{short_name[0]}.png"), img)


    if Normalisation_QC_target == "Adaptive Equalization":
        for filename in os.listdir(Target_QC_folder):

            img = io.imread(os.path.join(Target_QC_folder,filename))
            short_name = os.path.splitext(filename)

            img = exposure.equalize_adapthist(img, clip_limit=0.03)

            img = 255 * img # Now scale by 255
            img = img.astype(np.uint8)
            cv2.imwrite(os.path.join(Im_path, "B", "test", f"{short_name[0]}.png"), img)

    if Normalisation_QC_source == "None":
        for filename in os.listdir(Source_QC_folder):
            img = io.imread(os.path.join(Source_QC_folder,filename))
            short_name = os.path.splitext(filename)
            cv2.imwrite(os.path.join(Im_path, "A", "test", f"{short_name[0]}.png"), img)
            
    if Normalisation_QC_target == "None":
        for filename in os.listdir(Target_QC_folder):
            img = io.imread(os.path.join(Target_QC_folder,filename))
            short_name = os.path.splitext(filename)
            cv2.imwrite(os.path.join(Im_path, "B", "test", f"{short_name[0]}.png"), img)

def stack2im(path2stack, path2im, ph_normalisation=False):
    f = zis.CziFile(path2stack)
    stack = f.asarray()
    stack = stack.squeeze()
    file_name = os.path.basename(path2stack)
    file_name = os.path.splitext(file_name)
    print(f"{file_name} file loaded in python")
    os.makedirs(path2im, exist_ok=True)
    for t in range(len(stack)):
        if ph_normalisation:
            im = normalise_phc_timelapse(stack[t], keep_mean=False)
            # note that images are float 32 with negative values
            im = (im-np.min(im))/(np.max(im) - np.min(im))
        else:
            im = stack[t]
        cv2.imwrite(os.path.join(path2im, f"{file_name[0]}_{t:04d}.tif"), im)
    #Find image XY dimension
    Image_Y = im.shape[0]
    Image_X = im.shape[1]
    return min(Image_Y, Image_X)
    


def prepare_im_sequence(path2im, working_dir, pix2pix_code_dir, normalisation="Contrast stretching"):
    
    # Here we need to move the data to be analysed so that pix2pix can find them
    Saving_path_prediction= working_dir
    
    if os.path.exists(Saving_path_prediction):
        shutil.rmtree(Saving_path_prediction)
    os.makedirs(Saving_path_prediction, exist_ok=True)
    
    imageA_folder = os.path.join(Saving_path_prediction, "A")
    os.makedirs(imageA_folder, exist_ok=True)
    
    imageB_folder = os.path.join(Saving_path_prediction, "B")
    os.makedirs(imageB_folder, exist_ok=True)
    
    imageAB_folder = os.path.join(Saving_path_prediction, "AB")
    os.makedirs(imageAB_folder, exist_ok=True)
    
    testAB_Folder = os.path.join(imageAB_folder, "test")
    os.makedirs(testAB_Folder, exist_ok=True)
    
    testA_Folder = os.path.join(imageA_folder, "test")
    os.makedirs(testA_Folder, exist_ok=True)
    
    testB_Folder = os.path.join(imageB_folder, "test")
    os.makedirs(testB_Folder, exist_ok=True)
    
    # Normalise the image sequence with the pix2pix normalisation
    print(path2im)
    print(Saving_path_prediction)
    normalise_data(path2im, path2im, normalisation, normalisation, Saving_path_prediction)

    # Process normalised data for pix2pix to process it
    os.chdir(pix2pix_code_dir)
    !python3 pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A "$imageA_folder" --fold_B "$imageB_folder" --fold_AB "$imageAB_folder"
    print("Images ready to be processed")
    print(f'AB folder placed in {imageAB_folder}')
    
    return imageAB_folder
    
def process_im_sequence(pix2pix_code_dir, imageAB_folder, Prediction_model_name, Prediction_model_path, Result_folder, Nb_files_Data_folder, patch_size=1024, checkpoint="latest", nc=1):
    
    if not patch_size % 256 == 0:
      patch_size = ((int(patch_size / 256)) * 256)
      print (" Your image dimensions are not divisible by 256; therefore your images have now been resized to:",patch_size)
    
    if patch_size < 256:
      patch_size = 256

    
    os.chdir(pix2pix_code_dir)
    !python3 pytorch-CycleGAN-and-pix2pix/test.py --dataroot "$imageAB_folder" --name "$Prediction_model_name" --model pix2pix --no_dropout --preprocess scale_width --load_size $patch_size --crop_size $patch_size --results_dir "$Result_folder" --checkpoints_dir "$Prediction_model_path" --num_test $Nb_files_Data_folder --epoch $checkpoint --input_nc "$nc" --output_nc "$nc" --dataset_mode "aligned"
    print("Images processed already")


def frames2videos_pix2pix(path2im):
    path2real = os.path.join(path2im, "images_real_A")
    path2fake = os.path.join(path2im, "images_fake_B")
    
    files = os.listdir(path2real)
    
    video_names = [i.split("_0000_")[0] for i in files if i.__contains__("_0000_")]
    
    path2real_reconstructions = os.path.join(path2im, "videos_real_A")
    path2fake_reconstructions = os.path.join(path2im, "videos_fake_B")
    os.makedirs(path2real_reconstructions, exist_ok=True)
    os.makedirs(path2fake_reconstructions, exist_ok=True)
    for v in video_names:
        video = [j for j in files if j.__contains__(v)]
        video.sort()
        real_video = []
        fake_video = []
        
        for j in video:
            real_video.append(cv2.imread(os.path.join(path2real, j))[...,0])
            fake_filename = j.split("_real_A.png")[0] + "_fake_B.png"
            
            fake_video.append(cv2.imread(os.path.join(path2fake, fake_filename))[...,0])
        real_video = np.array(real_video)
        fake_video = np.array(fake_video)
        imsave(os.path.join(path2real_reconstructions, f"{v}.tif"), real_video)
        imsave(os.path.join(path2fake_reconstructions, f"{v}.tif"), fake_video)
        print(f"{v} processed")
        
def process_pix2pix(path2data, pix2pix_model_path, Result_folder, pix2pix_code_dir, working_dir, checkpoint="latest", normalisation="Contrast stretching", nc=1):
    folders = os.listdir(path2data)
    folders.sort
    print(folders)
    os.makedirs(Result_folder, exist_ok=True)
    path2imsequence = os.path.join(working_dir, "image_sequence")
    Prediction_model_name = os.path.basename(pix2pix_model_path)
    Prediction_model_path = os.path.dirname(pix2pix_model_path)

    if os.path.exists(path2imsequence):
        shutil.rmtree(path2imsequence)
    
    for i in range(len(folders)):
        f = folders[i]
        if f[0] != '.':
            if not f.__contains__('.'):

                process_pix2pix(os.path.join(path2data, f), pix2pix_model_path, os.path.join(Result_folder, f), 
                                pix2pix_code_dir, working_dir, checkpoint=checkpoint, 
                                normalisation=normalisation, nc=nc)
            elif f.__contains__('.czi'):
                print(f)
                # Convert the CZI stack into a normalised image sequence and save it in a general folder.
                patch_size = stack2im(os.path.join(path2data, f), path2imsequence, ph_normalisation=True)
                print(f'Images stored in {path2imsequence}')
            
                imageAB_folder = prepare_im_sequence(path2imsequence, os.path.join(working_dir, "prepared_data"), pix2pix_code_dir, normalisation=normalisation)
                
                Nb_files_Data_folder = len(os.listdir(os.path.join(imageAB_folder, "test")))+10
                
                process_im_sequence(pix2pix_code_dir, imageAB_folder, Prediction_model_name, Prediction_model_path,
                                    Result_folder, Nb_files_Data_folder, patch_size=patch_size, checkpoint=checkpoint, nc=nc)

                Checkpoint_name = "test_"+str(checkpoint)

                
                Prediction_results_folder = os.path.join(Result_folder, Prediction_model_name, Checkpoint_name, "images")
                
                Prediction_results_images = os.listdir(Prediction_results_folder)
                
                for f in Prediction_results_images:
                  if (f.endswith("_real_B.png")):
                    os.remove(Prediction_results_folder+"/"+f)
                
                
                os.makedirs(os.path.join(Prediction_results_folder + "_fake_B"),exist_ok=True)
                os.makedirs(os.path.join(Prediction_results_folder + "_real_A"),exist_ok=True)
                
                for f in os.listdir(Prediction_results_folder):
                    if f.endswith("fake_B.png"):
                        shutil.copy(os.path.join(Prediction_results_folder, f),
                                    os.path.join(Prediction_results_folder + "_fake_B", f))
                    elif f.endswith("real_A.png"):
                        shutil.copy(os.path.join(Prediction_results_folder, f),
                                    os.path.join(Prediction_results_folder + "_real_A", f))
                # Remove the images
                shutil.rmtree(Prediction_results_folder)

                if os.path.exists(path2imsequence):
                    shutil.rmtree(path2imsequence)

def digest_predictions(Result_folder):
    folders = os.listdir(Result_folder)
    folders.sort
    print(folders)
    if "images_fake_B" in folders:
        frames2videos_pix2pix(Result_folder)
    else:
        for f in folders:
            if f[0] != '.':
                digest_predictions(os.path.join(Result_folder, f))
    

In [3]:
#Here, we install libraries which are not already included in Colab.
#!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
import os
pix2pix_working_directory = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/PIX2PIX-MODELS/working_dir/"
pix2pix_code_dir = "/home/ocb/HardDrive_4TB/EGM/PHX/PhotoFITT/notebooks/deep-learning/pix2pix/"
#Result_folder = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/RESULTS/PHX_DATA"
Result_folder = "/media/ocb/OCB-Data2/PhotoFiTT/PIX2PIX/CHO-SYNCHRO-nuclei/"

pix2pix_model_path = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/PIX2PIX-MODELS/pix2pix_cho_selectedz_nuclei_ph_contrast_resized1024_18012024"
#pix2pix_model_path = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/PIX2PIX-MODELS/pix2pix_hela_selectedz_nuclei_ph_norm_contrast_resized1024_18042024/"

path2data = "/media/ocb/OCB-Data2/PhotoFiTT/RAW DATA/CHO-SYNCHRO-missing/"
process_pix2pix(path2data, pix2pix_model_path, Result_folder, pix2pix_code_dir, pix2pix_working_directory, checkpoint="latest", normalisation="Contrast stretching", nc=1)

#path2data = "/home/ocb/HardDrive_4TB/EGM/PHX/DATA/RAW"
#process_pix2pix(path2data, pix2pix_model_path, Result_folder, pix2pix_code_dir, pix2pix_working_directory, checkpoint="latest", normalisation="Contrast stretching", nc=1)

#path2data = "/media/ocb/OCB-Data 1/Phototoxicity/ITQB/Individual"
#process_pix2pix(path2data, pix2pix_model_path, Result_folder, pix2pix_code_dir, pix2pix_working_directory, checkpoint="latest", normalisation="Contrast stretching", nc=1)

digest_predictions(Result_folder)

['20230504_630_day', '20230505_475_day', '20230504_568_night_630-rename', '20230503_475_night']
['Individual']
['CHO_day_630__live-02-Scene-01-P1-A01.czi', 'CHO_day_630__live-02-Scene-02-P8-A01.czi', 'CHO_day_630__live-02-Scene-03-P7-A01.czi', 'CHO_day_630__live-02-Scene-04-P6-A01.czi', 'CHO_day_630__live-02-Scene-05-P10-A01.czi', 'CHO_day_630__live-02-Scene-06-P5-A01.czi', 'CHO_day_630__live-02-Scene-07-P2-A01.czi', 'CHO_day_630__live-02-Scene-08-P9-A01.czi', 'CHO_day_630__live-02-Scene-09-P3-A01.czi', 'CHO_day_630__live-02-Scene-10-P4-A01.czi', 'CHO_day_630__live-02-Scene-11-P6-A02.czi', 'CHO_day_630__live-02-Scene-12-P7-A02.czi', 'CHO_day_630__live-02-Scene-13-P5-A02.czi', 'CHO_day_630__live-02-Scene-14-P10-A02.czi', 'CHO_day_630__live-02-Scene-15-P1-A02.czi', 'CHO_day_630__live-02-Scene-16-P8-A02.czi', 'CHO_day_630__live-02-Scene-17-P2-A02.czi', 'CHO_day_630__live-02-Scene-18-P9-A02.czi', 'CHO_day_630__live-02-Scene-19-P4-A02.czi', 'CHO_day_630__live-02-Scene-20-P3-A02.czi', 'CHO_d

/tmp/ipykernel_31103/1761352285.py:183: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(os.path.join(path2real_reconstructions, f"{v}.tif"), real_video)
/tmp/ipykernel_31103/1761352285.py:184: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(os.path.join(path2fake_reconstructions, f"{v}.tif"), fake_video)


CHO_night_live-01-Scene-11-P1-A02 processed
CHO_night_live-01-Scene-24-P5-A03 processed
CHO_night_live-01-Scene-37-P8-A04 processed
CHO_night_live-01-Scene-59-P5-B03 processed
CHO_night_live-01-Scene-01-P1-A01 processed
CHO_night_live-01-Scene-02-P4-A01 processed
CHO_night_live-01-Scene-03-P2-A01 processed
CHO_night_live-01-Scene-04-P3-A01 processed
CHO_night_live-01-Scene-05-P5-A01 processed
CHO_night_live-01-Scene-06-P6-A01 processed
CHO_night_live-01-Scene-07-P7-A01 processed
CHO_night_live-01-Scene-08-P10-A01 processed
CHO_night_live-01-Scene-09-P9-A01 processed
CHO_night_live-01-Scene-10-P8-A01 processed
CHO_night_live-01-Scene-60-P6-B03 processed
CHO_night_live-01-Scene-61-P9-B02 processed
CHO_night_live-01-Scene-62-P10-B02 processed
CHO_night_live-01-Scene-63-P7-B02 processed
CHO_night_live-01-Scene-64-P2-B02 processed
CHO_night_live-01-Scene-65-P8-B02 processed
CHO_night_live-01-Scene-66-P6-B02 processed
CHO_night_live-01-Scene-67-P4-B02 processed
CHO_night_live-01-Scene-68-P1-

NotADirectoryError: [Errno 20] Not a directory: '/media/ocb/OCB-Data2/PhotoFiTT/PIX2PIX/CHO-SYNCHRO-nuclei/2022-07-26/videos_fake_B/CHO_20x_live-01-Scene-02-P10-A01.tif'

In [4]:
Result_folder = "/media/ocb/OCB-Data2/PhotoFiTT/PIX2PIX/CHO-SYNCHRO-nuclei/missing/"
digest_predictions(Result_folder)

['20230503_475_night', '20230504_568_night_630-rename', '20230505_475_day', '20230504_630_day']
['Individual']
['pix2pix_cho_selectedz_nuclei_ph_contrast_resized1024_18012024']
['test_latest']
['index.html', 'images_fake_B', 'images_real_A']


/tmp/ipykernel_31103/1761352285.py:183: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(os.path.join(path2real_reconstructions, f"{v}.tif"), real_video)
/tmp/ipykernel_31103/1761352285.py:184: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(os.path.join(path2fake_reconstructions, f"{v}.tif"), fake_video)


CHO_day_475__live-02-Scene-01-P3-A01 processed
CHO_day_475__live-02-Scene-02-P1-A01 processed
CHO_day_475__live-02-Scene-03-P8-A01 processed
CHO_day_475__live-02-Scene-04-P7-A01 processed
CHO_day_475__live-02-Scene-05-P6-A01 processed
CHO_day_475__live-02-Scene-06-P2-A01 processed
CHO_day_475__live-02-Scene-07-P10-A01 processed
CHO_day_475__live-02-Scene-08-P5-A01 processed
CHO_day_475__live-02-Scene-09-P4-A01 processed
CHO_day_475__live-02-Scene-10-P9-A01 processed
CHO_day_475__live-02-Scene-11-P5-A02 processed
CHO_day_475__live-02-Scene-12-P6-A02 processed
CHO_day_475__live-02-Scene-13-P1-A02 processed
CHO_day_475__live-02-Scene-14-P7-A02 processed
CHO_day_475__live-02-Scene-15-P2-A02 processed
CHO_day_475__live-02-Scene-16-P9-A02 processed
CHO_day_475__live-02-Scene-17-P8-A02 processed
CHO_day_475__live-02-Scene-18-P4-A02 processed
CHO_day_475__live-02-Scene-19-P3-A02 processed
CHO_day_475__live-02-Scene-20-P10-A02 processed
CHO_day_475__live-02-Scene-21-P5-A03 processed
CHO_day_475

In [ ]:
#Here, we install libraries which are not already included in Colab.
#!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
import os
pix2pix_working_directory = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/PIX2PIX-MODELS/working_dir/"
pix2pix_code_dir = "/home/ocb/HardDrive_4TB/EGM/PHX/PhotoFITT/notebooks/deep-learning/pix2pix/"
#Result_folder = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/RESULTS/PHX_DATA"
Result_folder = "/media/ocb/OCB-Data2/PhotoFiTT/PIX2PIX/CHO-SYNCHRO-nuclei"

pix2pix_model_path = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/PIX2PIX-MODELS/pix2pix_cho_selectedz_nuclei_ph_contrast_resized1024_18012024"
#pix2pix_model_path = "/home/ocb/HardDrive_4TB/EGM/MULTICHANNEL/PIX2PIX-MODELS/pix2pix_hela_selectedz_nuclei_ph_norm_contrast_resized1024_18042024/"

path2data = "/media/ocb/OCB-Data2/PhotoFiTT/RAW DATA/CHO-SYNCHRO"
process_pix2pix(path2data, pix2pix_model_path, Result_folder, pix2pix_code_dir, pix2pix_working_directory, checkpoint="latest", normalisation="Contrast stretching", nc=1)

#path2data = "/home/ocb/HardDrive_4TB/EGM/PHX/DATA/RAW"
#process_pix2pix(path2data, pix2pix_model_path, Result_folder, pix2pix_code_dir, pix2pix_working_directory, checkpoint="latest", normalisation="Contrast stretching", nc=1)

#path2data = "/media/ocb/OCB-Data 1/Phototoxicity/ITQB/Individual"
#process_pix2pix(path2data, pix2pix_model_path, Result_folder, pix2pix_code_dir, pix2pix_working_directory, checkpoint="latest", normalisation="Contrast stretching", nc=1)

digest_predictions(Result_folder)

In [ ]:
Result_folder = "/media/ocb/OCB-Data2/PhotoFiTT/PIX2PIX/CHO-SYNCHRO-nuclei"
digest_predictions(Result_folder)